In [1]:
!pip install faiss-cpu

!pip install mistralai 
!pip install openpyxl


In [ ]:
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 46.1 MB/s eta 0:00:0000:01


In [ ]:
import faiss
import numpy as np
import pandas as pd
import time
from mistralai import Mistral
import pickle

# 1. Définition de la requête
query_text = "taux de change dollar états unis"

# 2. Chargement de la clé OpenAI depuis un fichier sécurisé
from credentials.keys import MISTRALAI_API_KEY
client = Mistral(api_key=MISTRALAI_API_KEY)

# 3. Chargement du fichier Excel et préparation des données
file_path = "resultat_jointure.xlsx"  # Nom du fichier Excel
df = pd.read_excel(file_path)

#df=df.head(10)

df["title_fr"] = df["title_fr"].fillna("")
df["description_fr"] = df["description_fr"].fillna("")

def extract_periodicity(series_key):
    periodicity_mapping = {"D": "quotidien", "H": "hebdomadaire", "M": "mensuel", "A": "annuel", "Q": "trimestriel"}
    code = series_key.split(".")[1] if len(series_key.split(".")) > 1 else ""
    return periodicity_mapping.get(code, "")

df["periodicity"] = df["series_key"].apply(extract_periodicity)

documents = (df["description_fr"] + " " + df["title_fr"] + " " + df["periodicity"]).astype(str).tolist()
ids = df["series_key"].astype(str).tolist()

print("Taille des ids (fichier Excel) :", len(ids))

# 4. Création des embeddings et stockage dans FAISS
BATCH_SIZE = 200
total_tokens = 0

dim = 1024  # Taille des embeddings (doit être alignée avec le modèle utilisé)
index = faiss.IndexFlatL2(dim)  # Index FAISS pour la recherche par distance euclidienne

metadata = {}  # Stocker les ID et les métadonnées associées

print("Début de la génération des embeddings par lots de 200...")

for i in range(0, len(documents), BATCH_SIZE):
    batch_texts = documents[i : i + BATCH_SIZE]
    batch_ids = ids[i : i + BATCH_SIZE]
    
    response = client.embeddings.create(inputs=batch_texts, model="mistral-embed")
    
    batch_embeddings = np.array([resp.embedding for resp in response.data], dtype=np.float32)
    total_tokens += response.usage.total_tokens
    
    index.add(batch_embeddings)  # Ajout des embeddings à FAISS
    
    for doc_id, text in zip(batch_ids, batch_texts):
        metadata[doc_id] = {
            "title": df.loc[df["series_key"] == doc_id, "title_fr"].values[0],
            "description": df.loc[df["series_key"] == doc_id, "description_fr"].values[0],
            "periodicity": df.loc[df["series_key"] == doc_id, "periodicity"].values[0],
            "dataset_id": df.loc[df["series_key"] == doc_id, "dataset_id"].values[0],
        }

    print(f"Batch {i // BATCH_SIZE + 1} : {len(batch_texts)} embeddings générés et ajoutés dans FAISS.")
    time.sleep(5)

print(f"Total de tokens consommés : {total_tokens}")

# Sauvegarde de l'index FAISS et des métadonnées
faiss.write_index(index, "faiss_index_mistral.bin")
with open("metadata_mistral.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("Index FAISS et métadonnées sauvegardés.")

# 5. Générer l'embedding de la requête
response = client.embeddings.create(inputs=query_text, model="mistral-embed")
query_embedding = np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)

# 6. Recherche des voisins les plus proches
k = 10  # Nombre de résultats à récupérer
distances, indices = index.search(query_embedding, k)

# 7. Affichage des résultats
print("Phrase recherchée :", query_text)

for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
    if idx < len(metadata):  # Vérifier si l'index est valide
        doc_id = list(metadata.keys())[idx]
        meta_info = metadata[doc_id]
        print(f"Résultat {i+1}: {doc_id} (distance: {distance})")
        print(f"  - Titre: {meta_info['title']}")
        print(f"  - Description dataset: {meta_info['description']}")
        print(f"  - Périodicité: {meta_info['periodicity']}")
        print(f"  - Dataset: {meta_info['dataset_id']}")

# 8. Vérification d'une série particulière
specific_series_key = "EXR.D.USD.EUR.SP00.A"
if specific_series_key in metadata:
    specific_embedding = np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)
    distance = np.linalg.norm(query_embedding - specific_embedding)
    print(f"Distance entre '{specific_series_key}' et la requête : {distance}")
else:
    print(f"L'ID '{specific_series_key}' n'existe pas dans l'index FAISS.")


Taille des ids (fichier Excel) : 44371
Début de la génération des embeddings par lots de 200...
Batch 1 : 200 embeddings générés et ajoutés dans FAISS.
Batch 2 : 200 embeddings générés et ajoutés dans FAISS.
Batch 3 : 200 embeddings générés et ajoutés dans FAISS.
Batch 4 : 200 embeddings générés et ajoutés dans FAISS.
Batch 5 : 200 embeddings générés et ajoutés dans FAISS.
Batch 6 : 200 embeddings générés et ajoutés dans FAISS.
Batch 7 : 200 embeddings générés et ajoutés dans FAISS.
Batch 8 : 200 embeddings générés et ajoutés dans FAISS.
Batch 9 : 200 embeddings générés et ajoutés dans FAISS.
Batch 10 : 200 embeddings générés et ajoutés dans FAISS.
Batch 11 : 200 embeddings générés et ajoutés dans FAISS.
Batch 12 : 200 embeddings générés et ajoutés dans FAISS.
Batch 13 : 200 embeddings générés et ajoutés dans FAISS.
